In [ ]:
import pandas as pd
from pymongo import MongoClient
import re
import numpy as np
import time

In [ ]:
# Read csv files
dfKaggleReviews = pd.read_csv('Hotel_Reviews.csv')

dfKaggleReviews = dfKaggleReviews.drop(dfKaggleReviews[(dfKaggleReviews['Review_Total_Negative_Word_Counts'] == 0) & (
    dfKaggleReviews['Review_Total_Positive_Word_Counts'] == 0)].index)

# drop columns
dropping_columns = {'Hotel_Address', 'Additional_Number_of_Scoring', 'Average_Score', 'Reviewer_Nationality',
                    'Total_Number_of_Reviews', 'Total_Number_of_Reviews_Reviewer_Has_Given', 'Tags', 'days_since_review', 'Review_Date', 'Review_Total_Positive_Word_Counts', 'Review_Total_Negative_Word_Counts', 'lat', 'lng'}

dfKaggleReviews.drop(dropping_columns, inplace=True, axis=1)

In [ ]:
dfKaggleReviews2 = dfKaggleReviews.head(75000)

In [ ]:
dfKaggleReviews2

In [ ]:
def split_review(dfKaggle):
    df = pd.DataFrame()

    for index, row in dfKaggle.iterrows():
        df = df.append([[row.Hotel_Name, row.Reviewer_Score, row.Positive_Review,
                         1]], ignore_index=True)
        df = df.append([[row.Hotel_Name, row.Reviewer_Score, row.Negative_Review,
                         0]], ignore_index=True)
        # df.columns = cols

    return df

In [ ]:
dfKaggleReviews = split_review(dfKaggleReviews2)

In [ ]:
dfKaggleReviews.columns = ['Hotel_Name', 'Reviewer_Score', 'Review', 'label']

In [ ]:
def clean_review(text):
    if text is not None and text != '':
        # Removes special tekens/ spaties en zet alles naar lower case
        text = re.sub(r'[^\w\s]', '', str(text).lower().strip())

    return text

In [ ]:
dfKaggleReviews['Review'] = dfKaggleReviews['Review'].apply(
    clean_review)

In [ ]:
# Cleaning
dfKaggleReviews['Review'] = dfKaggleReviews['Review'].str.replace(
    'no negative', '')
dfKaggleReviews['Review'] = dfKaggleReviews['Review'].str.replace(
    'no positive', '')

In [ ]:
dfKaggleReviews.duplicated()
dfKaggleReviews = dfKaggleReviews.drop_duplicates(keep='first')

dfKaggleReviews['Review'].replace('', np.nan, inplace=True)
dfKaggleReviews.dropna(subset=['Review'], inplace=True)

dfKaggleReviews.info()

In [ ]:
# Database
client = MongoClient("localhost:27017")

# Collection name
db = client["hotel-reviews-database"]
db.clean_reviews.insert_many(dfKaggleReviews.to_dict('records'))